In [1]:
import os
from os.path import join, dirname, exists
import concurrent
import pandas as pd
from tqdm.notebook import tqdm
import pdfplumber
from datetime import datetime
import requests
import shutil
import sys

In [5]:
def check_exists(date):
    str_date = date.strftime('%Y%m%d')
    file_name = f"AQI_Bulletin_{str_date}.pdf"
    file_path = f"AQI_data/{file_name}"
    return exists(file_path), file_path, file_name

def download(date):
    file_exists, file_path, file_name = check_exists(date)
    if file_exists:
        return file_path

    os.makedirs("AQI_data", exist_ok=True)

    url = f"https://cpcb.nic.in//upload/Downloads/{file_name}"
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path, 'wb') as f:
            f.write(response.content)
        return file_path
    else:
        print(f"Failed to download {url} with status code {response.status_code}")
        return None

In [6]:
dates = pd.date_range('2016-01-01', datetime.today() - pd.Timedelta(days=1), freq='D')
# dates = pd.date_range('2024-01-01', '2024-02-01', freq='D')
print(dates), len(dates)

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2025-02-17', '2025-02-18', '2025-02-19', '2025-02-20',
               '2025-02-21', '2025-02-22', '2025-02-23', '2025-02-24',
               '2025-02-25', '2025-02-26'],
              dtype='datetime64[ns]', length=3345, freq='D')


(None, 3345)

In [7]:
with concurrent.futures.ThreadPoolExecutor(48) as executor:
    files = list(executor.map(download, dates))

Failed to download https://cpcb.nic.in//upload/Downloads/AQI_Bulletin_20160606.pdf with status code 404
Failed to download https://cpcb.nic.in//upload/Downloads/AQI_Bulletin_20170618.pdf with status code 404
Failed to download https://cpcb.nic.in//upload/Downloads/AQI_Bulletin_20171014.pdf with status code 404
Failed to download https://cpcb.nic.in//upload/Downloads/AQI_Bulletin_20250101.pdf with status code 404


In [8]:
print(len(files))
files = list(filter(None, files))
len(files)

3345


3341

In [9]:
def check_valid(value):
    if value is None:
        return False
    if value == "":
        return False
    return True

def process_pattern_1(table, i, key):
    # second line
    if (key is not None) and (not check_valid(table[i][0])):
        air_quality = table[i][2]
        return air_quality
    
    # first line
    if check_valid(table[i][0]):
        key = table[i][0]
        aqi = int(table[i][4])
        pollutant = table[i][5]
        air_quality = process_pattern_1(table, i+1, key)
        return {key: {"AQI": aqi, "Pollutant": pollutant, "Air Quality": air_quality, "Based on number of monitoring stations": None}}
    
def process_pattern_2(table, i, key):
    # second line
    try:
        if (key is not None) and (not check_valid(table[i][0])):
            air_quality = table[i][2]
            return air_quality
    except Exception as e:
        print(table[i-1])
        raise e
    
    # first line
    if check_valid(table[i][0]):
        try:
            key = table[i][0]
            if check_valid(table[i][1]):
                air_quality = table[i][1]
                need_for_second_line = False
            else:
                need_for_second_line = True
            aqi = int(table[i][4])
            pollutant = table[i][5] # p2
            n_stations = table[i][6]
        except Exception as e:
            print(table[i])
            print(table)
            raise e
        if need_for_second_line:
            air_quality = process_pattern_2(table, i+1, key)
        return {key: {"AQI": aqi, "Pollutant": pollutant, "Air Quality": air_quality, "Based on number of monitoring stations": n_stations}}
    
def process_pattern_3(table, i, key):
    # second line
    if (key is not None) and (not check_valid(table[i][0])):
        air_quality = table[i][2]
        return air_quality
    
    # first line
    if check_valid(table[i][0]):
        key = table[i][0]
        if check_valid(table[i][1]):
            air_quality = table[i][1]
            second_line_needed = False
        else:
            second_line_needed = True
        aqi = int(table[i][2])
        pollutant = table[i][3]
        n_stations = table[i][4]
        if second_line_needed:
            air_quality = process_pattern_3(table, i+1, key)
        return {key: {"AQI": aqi, "Pollutant": pollutant, "Air Quality": air_quality, "Based on number of monitoring stations": n_stations}}

def process_pattern_4(table, i, key):
    # ['S.No', 'City', 'Air Quality', 'Index Value', 'Prominent Pollutant', 'Based on Number\nof Monitoring\nStations'], ['1', 'Agra', 'Moderate', '138', 'PM\n2.5', '1'], [None, None, '', None, None, None], ['2', 'Ahmedabad', 'Satisfactory', '77', 'PM\n10', '1'], [None, None, '', None, None, None], ['3', 'Aizawl', 'Satisfactory', '53', 'PM\n2.5', '1'], [None, None, '', None, None, None], ['4', 'Ajmer', 'Satisfa
    # # invalid line
    # if (key is not None) and (not check_valid(table[i][0])):
    #     air_quality = table[i][2]
    #     return air_quality
    
    # first line
    if check_valid(table[i][0]):
        key = table[i][1]
        air_quality = table[i][2]
        aqi = int(table[i][3].split("\n")[0])
        pollutant = table[i][4]
        n_stations = table[i][5]
        return {key: {"AQI": aqi, "Pollutant": pollutant, "Air Quality": air_quality, "Based on number of monitoring stations": n_stations}}
    

def process_table(table, start):
    data_dict = {}
    if table[0] == ['City', 'Air Quality', None, None, 'Index Value', 'Prominent\nPollutant']:
        table = table[1:]
        for i in range(len(table)):
            data = process_pattern_1(table, i, None)
            if data:
                data_dict.update(process_pattern_1(table, i, None))
                
        df = pd.DataFrame(data_dict).T
        df.index.name = "City"
        df.reset_index(inplace=True, drop=False)
        return df, None
    elif (table[0] == ['City', 'Air Quality', None, None, 'Index Value', 'Prominent\nPollutant', 'Based on number of\nmonitoring stations']) or (table[0] == ['City', 'Air Quality', None, None, 'Index Value', None, None, 'Prominent\nPollutant', 'Based on\nnumber of\nstations']) or (table[0] == ['City', 'Air Quality', None, None, 'Index Value', 'Prominent\nPollutant', 'Based on number of\nmonitoring stations']) or (table[0] == ['City', 'Air Quality', None, None, 'Index Value', 'Prominent\nPollutant', 'Based on\nnumber of\nmonitoring\nstations']) or (table[0] == ['City', 'Air Quality', None, None, None, None, 'Index Value', None, None, 'Prominent\nPollutant', 'Based on number of\nstations']) or (table[0] == ['City', 'Air Quality', None, None, None, None, 'Index Value', 'Prominent\nPollutant', 'Based on\nnumber of\nmonitoring\nstations']) or (table[0] == ['City', 'Air Quality', None, None, 'Index\nValue', 'Prominent\nPollutant', 'Based on number of\nmonitoring stations']) or (table[0] == ['City', 'Air Quality', None, None, None, None, 'Index Value', 'Prominent\nPollutant', 'Based on\nnumber of\nstations']):
        table = table[1:]
        for i in range(len(table)):
            data = process_pattern_2(table, i, None)
            if data:
                data_dict.update(process_pattern_2(table, i, None))
                
        df = pd.DataFrame(data_dict).T
        df.index.name = "City"
        df.reset_index(inplace=True, drop=False)
        return df, None
    elif table[0] == ['City', 'Air Quality', 'Index Value', 'Prominent\nPollutant', 'Based on number of\nmonitoring stations']:
        # print("Pattern 3")
        table = table[1:]
        for i in range(len(table)):
            data = process_pattern_3(table, i, None)
            if data:
                data_dict.update(process_pattern_3(table, i, None))
        df = pd.DataFrame(data_dict).T
        df.index.name = "City"
        df.reset_index(inplace=True, drop=False)
        return df, None
    elif (table[0] == ['S.No', 'City', 'Air Quality', 'Index Value', 'Prominent Polluta\nnt', 'Based on number\nof monitoring\nstations']) or (table[0] == ['S.No', 'City', 'Air Quality', 'Index Value', 'Prominent Pollutant', 'Based on Number\nof Monitoring\nStations']) or (table[0] == ['S.No', 'City', 'Air Quality', 'Index\nValue', 'Prominent Pollutant', 'No. of Stations\nParticipated/\nTotal Stations']):
        # print("Pattern 4")
        table = table[1:]
        for i in range(len(table)):
            data = process_pattern_4(table, i, None)
            if data:
                data_dict.update(process_pattern_4(table, i, None))
        
        df = pd.DataFrame(data_dict).T
        df.index.name = "City"
        df.reset_index(inplace=True, drop=False)
        return df, None
    elif (table[0] == ['Good', 'Minimal impact']) or (table[0] == ['Good', 'Minimal Impact']) or (table[0] == ['AQI', 'Category', 'Color Code', 'Possible Health Impacts']):
        # print("Not a data table")
        return None, None
    else:
        print(table)
        raise ValueError("Table pattern not recognized")

def process_file(date):
    folders = ["AQI_data", "AQI_data_csv"]

    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
    file_exists, file_path, file_name = check_exists(date)
    if not file_exists:
        print(f"File {file_name} does not exist")
        return None
    save_path = file_name.replace(".pdf",".csv")
    if exists(f"AQI_data_csv/{save_path}"):
        try:
            pd.read_csv(f"AQI_data_csv/{save_path}")
            # print(f"File {save_path} already exists")
            return None
        except Exception as e:
            print(f"File {save_path} is corrupted and will be overwritten")
    
    tables = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                tables.append(table)
    try:
        assert len(tables) > 0, f"No tables found in {file_path}"
    except AssertionError:
        print(f"No tables found in {file_path}")
        return None

    df_list = []
    for table in tables:
        try:
            df, _ = process_table(table, 0)
            if df is not None:
                df_list.append(df)
        except Exception as e:
            print(f"Ignoring a table for {file_name}")
            # print(table)
            print("Error message:", e)
    
    if len(df_list) == 0:
        print(f"No valid tables found in {file_name}")
        return None
    
    df = pd.concat(df_list, ignore_index=True)
    df['Date'] = date
    df.to_csv(f"AQI_data_csv/{save_path}", index=False)

from joblib import Parallel, delayed

# dfs = {}
# for file_path in tqdm(files[1000:]):
    # print(file_path)
    # df = process_file(file_path)
    # dfs[file_path] s= df
# print(dates[15:16])
_ = Parallel(48)(delayed(process_file)(file_path) for file_path in tqdm(dates))

  0%|          | 0/3345 [00:00<?, ?it/s]

## Creating Merged DataFrame

In [10]:
import os
import pandas as pd

def merge_csv_files(folder_path, output_file):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    if not csv_files:
        print("No CSV files found in the folder.")
        return

    df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
    merged_df = pd.concat(df_list, ignore_index=True)

    merged_df.to_csv(output_file, index=False)
    print(f"Merged CSV saved as {output_file}")

# Example usage
folder_path = "AQI_data_csv"
output_file = "AQI_data_csv/merged.csv"
merge_csv_files(folder_path, output_file)

Merged CSV saved as AQI_data_csv/merged.csv


## Postprocessing

In [11]:
merged_df = pd.read_csv("AQI_data_csv/merged.csv")
len(merged_df)

397732

In [12]:
city_mapping = {
    "Amaravati": "Amravati",
    "Asanol": "Asansol",
    "Greater Noida": "Greater_Noida",
    "GandhiNagar": "Gandhinagar",
    "Gurgaon": "Gurugram",
    "Coimbatore": "Coimbtore",
    "Kalaburagi": "Kalaburgi",
    "Kurukshetra": "Kurushketra",
    "Loni_Dehat": "Loni_Ghaziabad",
    "Madikeri": "Medikeri",
    "Manglore": "Mangalore",
    "Pimpri Chinchwad": "Pimpri-Chinchwad",
    "Tumakuru": "Tumidih",
    "Tirumala": "Tirupati",
    "Tiruppur": "Tirupur",
    "Yamuna Nagar": "Yamunanagar",
    "vellore": "Vellore"  # duplicate, can map to itself or be handled separately
}
def replace_it(x):
    x = x.strip().replace("\n","")
    if x in city_mapping:
        return city_mapping[x]
    else:
        return x

merged_df['City'] = merged_df['City'].apply(lambda x: replace_it(x))
merged_df = merged_df[merged_df.City != "Aurangabad"]
print(merged_df['City'].value_counts().sort_index().index.tolist())

['Agartala', 'Agra', 'Ahmedabad', 'Ahmednagar', 'Aizawl', 'Ajmer', 'Akola', 'Alwar', 'Ambala', 'Amravati', 'Amritsar', 'Anantapur', 'Angul', 'Ankleshwar', 'Araria', 'Ariyalur', 'Arrah', 'Asansol', 'Aurangabad (Bihar)', 'Aurangabad(Maharashtra)', 'Baddi', 'Badlapur', 'Bagalkot', 'Baghpat', 'Bahadurgarh', 'Balasore', 'Ballabgarh', 'Banswara', 'Baran', 'Barbil', 'Bareilly', 'Baripada', 'Barmer', 'Barrackpore', 'Bathinda', 'Begusarai', 'Belapur', 'Belgaum', 'Bengaluru', 'Bettiah', 'Bhagalpur', 'Bharatpur', 'Bhilai', 'Bhilwara', 'Bhiwadi', 'Bhiwandi', 'Bhiwani', 'Bhopal', 'Bhubaneswar', 'Bidar', 'Bihar Sharif', 'Bikaner', 'Bilaspur', 'Bileipada', 'Boisar', 'Brajrajnagar', 'Bulandshahr', 'Bundi', 'Buxar', 'Byasanagar', 'Byrnihat', 'Chamarajanagar', 'Chandigarh', 'Chandrapur', 'Charkhi Dadri', 'Chengalpattu', 'Chennai', 'Chhal', 'Chhapra', 'Chikkaballapur', 'Chikkamagaluru', 'Chittoor', 'Chittorgarh', 'Churu', 'Coimbtore', 'Cuddalore', 'Cuttack', 'Damoh', 'Darbhanga', 'Dausa', 'Davanagere', '

In [13]:
city_to_state = {
    'Agartala': 'Tripura', 'Agra': 'Uttar Pradesh', 'Ahmedabad': 'Gujarat', 'Ahmednagar': 'Maharashtra',
    'Aizawl': 'Mizoram', 'Ajmer': 'Rajasthan', 'Akola': 'Maharashtra', 'Alwar': 'Rajasthan', 
    'Ambala': 'Haryana', 'Amravati': 'Maharashtra', 'Amritsar': 'Punjab', 'Anantapur': 'Andhra Pradesh', 
    'Angul': 'Odisha', 'Ankleshwar': 'Gujarat', 'Araria': 'Bihar', 'Ariyalur': 'Tamil Nadu', 
    'Arrah': 'Bihar', 'Asansol': 'West Bengal', 'Aurangabad (Bihar)': 'Bihar', 
    'Aurangabad(Maharashtra)': 'Maharashtra', 'Baddi': 'Himachal Pradesh', 'Badlapur': 'Maharashtra', 
    'Bagalkot': 'Karnataka', 'Baghpat': 'Uttar Pradesh', 'Bahadurgarh': 'Haryana', 'Balasore': 'Odisha', 
    'Ballabgarh': 'Haryana', 'Banswara': 'Rajasthan', 'Baran': 'Rajasthan', 'Barbil': 'Odisha', 
    'Bareilly': 'Uttar Pradesh', 'Baripada': 'Odisha', 'Barmer': 'Rajasthan', 'Barrackpore': 'West Bengal', 
    'Bathinda': 'Punjab', 'Begusarai': 'Bihar', 'Belapur': 'Maharashtra', 'Belgaum': 'Karnataka', 
    'Bengaluru': 'Karnataka', 'Bettiah': 'Bihar', 'Bhagalpur': 'Bihar', 'Bharatpur': 'Rajasthan', 
    'Bhilai': 'Chhattisgarh', 'Bhilwara': 'Rajasthan', 'Bhiwadi': 'Rajasthan', 'Bhiwandi': 'Maharashtra', 
    'Bhiwani': 'Haryana', 'Bhopal': 'Madhya Pradesh', 'Bhubaneswar': 'Odisha', 'Bidar': 'Karnataka', 
    'Bihar Sharif': 'Bihar', 'Bikaner': 'Rajasthan', 'Bilaspur': 'Chhattisgarh', 'Bileipada': 'Odisha', 
    'Boisar': 'Maharashtra', 'Brajrajnagar': 'Odisha', 'Bulandshahr': 'Uttar Pradesh', 'Bundi': 'Rajasthan', 
    'Buxar': 'Bihar', 'Byasanagar': 'Odisha', 'Byrnihat': 'Meghalaya', 'Chamarajanagar': 'Karnataka', 
    'Chandigarh': 'Chandigarh', 'Chandrapur': 'Maharashtra', 'Charkhi Dadri': 'Haryana', 
    'Chengalpattu': 'Tamil Nadu', 'Chennai': 'Tamil Nadu', 'Chhal': 'Chhattisgarh', 'Chhapra': 'Bihar', 
    'Chikkaballapur': 'Karnataka', 'Chikkamagaluru': 'Karnataka', 'Chittoor': 'Andhra Pradesh', 
    'Chittorgarh': 'Rajasthan', 'Churu': 'Rajasthan', 'Coimbtore': 'Tamil Nadu', 'Cuddalore': 'Tamil Nadu', 
    'Cuttack': 'Odisha', 'Damoh': 'Madhya Pradesh', 'Darbhanga': 'Bihar', 'Dausa': 'Rajasthan', 
    'Davanagere': 'Karnataka', 'Dehradun': 'Uttarakhand', 'Delhi': 'Delhi', 'Dewas': 'Madhya Pradesh', 
    'Dhanbad': 'Jharkhand', 'Dharuhera': 'Haryana', 'Dharwad': 'Karnataka', 'Dholpur': 'Rajasthan', 
    'Dhule': 'Maharashtra', 'Dindigul': 'Tamil Nadu', 'Dungarpur': 'Rajasthan', 'Durgapur': 'West Bengal', 
    'Eloor': 'Kerala', 'Ernakulam': 'Kerala', 'Faridabad': 'Haryana', 'Fatehabad': 'Haryana', 
    'Firozabad': 'Uttar Pradesh', 'Gadag': 'Karnataka', 'Gandhinagar': 'Gujarat', 'Gangtok': 'Sikkim', 
    'Gaya': 'Bihar', 'Ghaziabad': 'Uttar Pradesh', 'Gorakhpur': 'Uttar Pradesh', 'Greater_Noida': 'Uttar Pradesh', 
    'Gummidipoondi': 'Tamil Nadu', 'Gurugram': 'Haryana', 'Guwahati': 'Assam', 'Gwalior': 'Madhya Pradesh', 
    'Hajipur': 'Bihar', 'Haldia': 'West Bengal', 'Hanumangarh': 'Rajasthan', 'Hapur': 'Uttar Pradesh', 
    'Hassan': 'Karnataka', 'Haveri': 'Karnataka', 'Hisar': 'Haryana', 'Hosur': 'Tamil Nadu', 'Howrah': 'West Bengal', 
    'Hubballi': 'Karnataka', 'Hyderabad': 'Telangana', 'Imphal': 'Manipur', 'Indore': 'Madhya Pradesh', 
    'Jabalpur': 'Madhya Pradesh', 'Jaipur': 'Rajasthan', 'Jaisalmer': 'Rajasthan', 'Jalandhar': 'Punjab', 
    'Jalgaon': 'Maharashtra', 'Jalna': 'Maharashtra', 'Jalore': 'Rajasthan', 'Jhalawar': 'Rajasthan', 
    'Jhansi': 'Uttar Pradesh', 'Jharsuguda': 'Odisha', 'Jhunjhunu': 'Rajasthan', 'Jind': 'Haryana', 
    'Jodhpur': 'Rajasthan', 'Jorapokhar': 'Jharkhand', 'Kadapa': 'Andhra Pradesh', 'Kaithal': 'Haryana', 
    'Kalaburgi': 'Karnataka', 'Kalyan': 'Maharashtra', 'Kanchipuram': 'Tamil Nadu', 'Kannur': 'Kerala', 
    'Kanpur': 'Uttar Pradesh', 'Karauli': 'Rajasthan', 'Karnal': 'Haryana', 'Karur': 'Tamil Nadu', 
    'Karwar': 'Karnataka', 'Kashipur': 'Uttarakhand', 'Katihar': 'Bihar', 'Katni': 'Madhya Pradesh', 
    'Keonjhar': 'Odisha', 'Khanna': 'Punjab', 'Khurja': 'Uttar Pradesh', 'Kishanganj': 'Bihar', 
    'Kochi': 'Kerala', 'Kohima': 'Nagaland', 'Kolar': 'Karnataka', 'Kolhapur': 'Maharashtra', 
    'Kolkata': 'West Bengal', 'Kollam': 'Kerala', 'Koppal': 'Karnataka', 'Korba': 'Chhattisgarh', 
    'Kota': 'Rajasthan', 'Kozhikode': 'Kerala', 'Kunjemura': 'Jharkhand', 'Kurushketra': 'Haryana', 
    'Latur': 'Maharashtra', 'Loni_Ghaziabad': 'Uttar Pradesh', 'Lucknow': 'Uttar Pradesh', 'Ludhiana': 'Punjab', 
    'Madurai': 'Tamil Nadu', 'Mahad': 'Maharashtra', 'Maihar': 'Madhya Pradesh', 'Malegaon': 'Maharashtra', 
    'Mandi Gobindgarh': 'Punjab', 'Mandideep': 'Madhya Pradesh', 'Mandikhera': 'Haryana', 'Manesar': 'Haryana', 
    'Mangalore': 'Karnataka', 'Manguraha': 'Bihar', 'Medikeri': 'Karnataka', 'Meerut': 'Uttar Pradesh', 
    'Milupara': 'Chhattisgarh', 'Mira-Bhayandar': 'Maharashtra', 'Moradabad': 'Uttar Pradesh', 
    'Motihari': 'Bihar', 'Mumbai': 'Maharashtra', 'Munger': 'Bihar', 'Muzaffarnagar': 'Uttar Pradesh', 
    'Muzaffarpur': 'Bihar', 'Mysuru': 'Karnataka', 'NOIDA': 'Uttar Pradesh', 'Nagaon': 'Assam', 
    'Nagapattinam': 'Tamil Nadu', 'Nagaur': 'Rajasthan', 'Nagpur': 'Maharashtra', 'Naharlagun': 'Arunachal Pradesh', 
    'Nalbari': 'Assam', 'Nanded': 'Maharashtra', 'Nandesari': 'Gujarat', 'Narnaul': 'Haryana', 'Nashik': 'Maharashtra',
    'Navi Mumbai': 'Maharashtra',
    'Nayagarh': 'Odisha',
    'Noida': 'Uttar Pradesh',
    'Ooty': 'Tamil Nadu',
    'Pali': 'Rajasthan',
    'Palkalaiperur': 'Tamil Nadu',
    'Palwal': 'Haryana',
    'Panchkula': 'Haryana',
    'Panipat': 'Haryana',
    'Parbhani': 'Maharashtra',
    'Pathardih': 'Jharkhand',
    'Patiala': 'Punjab',
    'Patna': 'Bihar',
    'Pimpri-Chinchwad': 'Maharashtra',
    'Pithampur': 'Madhya Pradesh',
    'Pratapgarh': 'Rajasthan',
    'Prayagraj': 'Uttar Pradesh',
    'Puducherry': 'Puducherry',
    'Pudukottai': 'Tamil Nadu',
    'Pune': 'Maharashtra',
    'Purnia': 'Bihar',
    'Raichur': 'Karnataka',
    'Raipur': 'Chhattisgarh',
    'Rairangpur': 'Odisha',
    'Rajamahendravaram': 'Andhra Pradesh',
    'Rajgir': 'Bihar',
    'Rajsamand': 'Rajasthan',
    'Ramanagara': 'Karnataka',
    'Ramanathapuram': 'Tamil Nadu',
    'Ranipet': 'Tamil Nadu',
    'Ratlam': 'Madhya Pradesh',
    'Rishikesh': 'Uttarakhand',
    'Rohtak': 'Haryana',
    'Rourkela': 'Odisha',
    'Rupnagar': 'Punjab',
    'Sagar': 'Madhya Pradesh',
    'Saharsa': 'Bihar',
    'Salem': 'Tamil Nadu',
    'Samastipur': 'Bihar',
    'Sangli': 'Maharashtra',
    'Sasaram': 'Bihar',
    'Satna': 'Madhya Pradesh',
    'Sawai Madhopur': 'Rajasthan',
    'Shillong': 'Meghalaya',
    'Shivamogga': 'Karnataka',
    'Sikar': 'Rajasthan',
    'Silchar': 'Assam',
    'Siliguri': 'West Bengal',
    'Singrauli': 'Madhya Pradesh',
    'Sirohi': 'Rajasthan',
    'Sirsa': 'Haryana',
    'Sivasagar': 'Assam',
    'Siwan': 'Bihar',
    'Solapur': 'Maharashtra',
    'Sonipat': 'Haryana',
    'Sri Ganganagar': 'Rajasthan',
    'Sri Vijaya Puram': 'Andaman and Nicobar',
    'Srinagar': 'Jammu and Kashmir',
    'Suakati': 'Odisha',
    'Surat': 'Gujarat',
    'Talcher': 'Odisha',
    'Tensa': 'Odisha',
    'Thane': 'Maharashtra',
    'Thanjavur': 'Tamil Nadu',
    'Thiruvananthapuram': 'Kerala',
    'Thoothukudi': 'Tamil Nadu',
    'Thrissur': 'Kerala',
    'Tiruchirappalli': 'Tamil Nadu',
    'Tirunelveli': 'Tamil Nadu',
    'Tirupati': 'Andhra Pradesh',
    'Tirupur': 'Tamil Nadu',
    'Tonk': 'Rajasthan',
    'Tumidih': 'Chhattisgarh',
    'Udaipur': 'Rajasthan',
    'Udupi': 'Karnataka',
    'Ujjain': 'Madhya Pradesh',
    'Ulhasnagar': 'Maharashtra',
    'Vapi': 'Gujarat',
    'Varanasi': 'Uttar Pradesh', 
    'Vatva': 'Gujarat', 'Vellore': 'Tamil Nadu',
    'Vijayapura': 'Karnataka',
    'Vijayawada': 'Andhra Pradesh',
    'Virar': 'Maharashtra',
    'Virudhunagar': 'Tamil Nadu',
    'Visakhapatnam': 'Andhra Pradesh',
    'Vrindavan': 'Uttar Pradesh',
    'Yadgir': 'Karnataka',
    'Yamunanagar': 'Haryana'
}
merged_df['State'] = merged_df['City'].apply(lambda x: city_to_state[x])
merged_df['State'].value_counts().sort_index()

State
Andaman and Nicobar        6
Andhra Pradesh         11546
Arunachal Pradesh        614
Assam                   5099
Bihar                  28633
Chandigarh              1980
Chhattisgarh            5357
Delhi                   3330
Gujarat                12195
Haryana                50177
Himachal Pradesh        1022
Jammu and Kashmir        822
Jharkhand               2076
Karnataka              35248
Kerala                 11549
Madhya Pradesh         31326
Maharashtra            39193
Manipur                  790
Meghalaya               1956
Mizoram                 1535
Nagaland                1398
Odisha                 12363
Puducherry              1433
Punjab                 19676
Rajasthan              37729
Sikkim                   812
Tamil Nadu             14080
Telangana               3322
Tripura                 1442
Uttar Pradesh          41800
Uttarakhand             2156
West Bengal            15406
Name: count, dtype: int64

In [14]:
# merged_df['Based on number of monitoring stations'] = merged_df['Based on number of monitoring stations'].apply(lambda x: int(x.split("/")[0].split("#")[0].split("\n")[0]))
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df.head(2)

,City,AQI,Pollutant,Air Quality,Based on number of monitoring stations,Date,State
0,Agra,417,PM\n2.5,Severe,1,2016-01-01,Uttar Pradesh
1,Bengaluru,95,"PM , PM\n2.5 10",Satisfactory,5,2016-01-01,Karnataka


In [15]:
len(merged_df)

396071

In [16]:
merged_df['Date_City'] = merged_df['Date'].astype(str) + "_" + merged_df['City']
print(len(merged_df))
merged_df = merged_df.drop_duplicates(subset=['Date_City'], keep='first')
print(len(merged_df))

396071
395213


In [17]:
merged_df.set_index(["Date", "City"]).to_xarray().to_dataframe().to_parquet("AQI_data.parquet")